In [55]:
# !pip install  apache_beam
# !pip install timezonefinder 

In [53]:
import warnings
warnings.filterwarnings('ignore')

In [54]:
import apache_beam as beam
import csv

ファイルからid,緯度経度を抽出

In [56]:
with beam.Pipeline('DirectRunner') as pipeline:

  # データの読み込み、フィルタリング
  airports=(pipeline
            | beam.io.ReadFromText('airports.csv.gz')
            | beam.Map(lambda line:next(csv.reader([line])))
            # 空港のid,緯度、経度のcolumnのみ取得
            | beam.Map(lambda fields : (fields[0],(fields[21],fields[26])))
           )

  # データのパース、書き込み
  airports | beam.Map(lambda (airport,data):'{},{}'.format(airport,','.join(data))) | beam.io.textio.WriteToText('extracted_airports')

  pipeline.run()

緯度経度からtimezoneを変換

In [57]:
def add_timezone(lat,lon):
  try:
    # 内部でimport するのはDataflowだと推奨されている
    # https://cloud.google.com/dataflow/faq
    import timezonefinder
    tf=timezonefinder.TimezoneFinder()
    tz=tf.timezone_at(lng=float(lon),lat=float(lat))
    
    if tz is None:
      tz='UTC'
    return (lat,lon,tz)
  
  except ValueError:
    return(lat,lon,'TIMEZONE')

In [46]:
with beam.Pipeline('DirectRunner') as pipeline:

  # データの読み込み、フィルタリング
  airports=(pipeline
            | beam.io.ReadFromText('airports.csv.gz')
            # headerは飛ばす
            | beam.Map(lambda line:next(csv.reader([line])))
            # 空港のid,緯度、経度のcolumnのみ取得し、tzに変換
            | beam.Map(lambda fields : (fields[0],add_timezone(fields[21],fields[26])))
           )

  # データのパース、書き込み
  airports | beam.Map(lambda (airport,data):'{},{}'.format(airport,','.join(data))) | beam.io.textio.WriteToText('extracted_airports')

  pipeline.run()

/usr/local/envs/py2env/lib/python2.7/site-packages/simplejson/encoder.py:286: DeprecationWarning: Interpreting naive datetime as local 2018-05-19 08:51:57.013660. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [61]:
%storage --h

usage: storage [-h] {copy,create,delete,list,read,view,write} ...

Execute various storage-related operations. Use "%storage <command> -h" for
help on a specific command.

positional arguments:
  {copy,create,delete,list,read,view,write}
                        commands
    copy                Copy one or more GCS objects to a different location.
    create              Create one or more GCS buckets.
    delete              Delete one or more GCS buckets or objects.
    list                List buckets in a project, or contents of a bucket.
    read                Read the contents of a storage object into a Python
                        variable.
    view                View the contents of a storage object.
    write               Write the value of a Python variable to a storage
                        object.

optional arguments:
  -h, --help            show this help message and exit
None


In [58]:
# GSに予め保存した離着陸データを取得
%storage list

Bucket,Created
gs://csv-data-bucket,2018-04-15 11:56:35.082000+00:00
gs://data-science-on-the-gcp.appspot.com,2018-04-15 11:54:22.540000+00:00
gs://staging.data-science-on-the-gcp.appspot.com,2018-04-15 11:54:22.648000+00:00


In [68]:
%storage list -b  gs://csv-data-bucket

Name,Type,Size,Updated
flights/1.csv,text/csv,77169964,2018-04-15 12:01:38.031000+00:00
flights/3.csv,text/csv,83821611,2018-04-15 12:01:37.414000+00:00


In [80]:
%storage view -o gs://csv-data-bucket/flights/1.csv 

u'"FL_DATE","UNIQUE_CARRIER","AIRLINE_ID","CARRIER","FL_NUM","ORIGIN_AIRPORT_ID","ORIGIN_AIRPORT_SEQ_ID","ORIGIN_CITY_MARKET_ID","ORIGIN","DEST_AIRPORT_ID","DEST_AIRPORT_SEQ_ID","DEST_CITY_MARKET_ID","DEST","CRS_DEP_TIME","DEP_TIME","DEP_DELAY","TAXI_OUT","WHEELS_OFF","WHEELS_ON","TAXI_IN","CRS_ARR_TIME","ARR_TIME","ARR_DELAY","CANCELLED","CANCELLATION_CODE","DIVERTED","DISTANCE",\n2017-01-01,"AA",19805,"AA","1766",13930,1393004,30977,"ORD",11298,1129804,30194,"DFW","0710","0705",-5.00,12.00,"0717","0918",14.00,"0948","0932",-16.00,0.00,"",0.00,802.00,\n2017-01-01,"AA",19805,"AA","1767",12889,1288903,32211,"LAS",11298,1129804,30194,"DFW","1235","1235",0.00,11.00,"1246","1703",14.00,"1718","1717",-1.00,0.00,"",0.00,1055.00,\n2017-01-01,"AA",19805,"AA","1768",11278,1127803,30852,"DCA",13303,1330303,32467,"MIA","1020","1015",-5.00,20.00,"1035","1246",7.00,"1313","1253",-20.00,0.00,"",0.00,919.00,\n2017-01-01,"AA",19805,"AA","1769",15304,1530402,33195,"TPA",13303,1330303,32467,"MIA","0730"